### Load Data

In [1]:
model_name = 'model_siamese_diffpool1'
num_epochs = 30

In [2]:
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.loader import DenseDataLoader #To make use of this data loader, all graph attributes in the dataset need to have the same shape. In particular, this data loader should only be used when working with dense adjacency matrices.
from torch_geometric.nn import DenseGCNConv, dense_diff_pool
from torch_geometric.utils import dense_to_sparse
import math
import numpy as np

In [3]:
data_dir_1 = 'C:/Users/david/pyproj/pyg/adl/patch_label_1'
data_dir_0 = 'C:/Users/david/pyproj/pyg/adl/patch_label_0'
#data_dir_1 = 'C:/Users/thoma/OneDrive - ZHAW/ProteinSurfaces-DESKTOP-AQ00763/patch_label_1'
#data_dir_0 = 'C:/Users/thoma/OneDrive - ZHAW/ProteinSurfaces-DESKTOP-AQ00763/patch_label_0'

In [4]:
from c_PatchDataset import PatchDataset
dataset = PatchDataset(data_dir_label_0 = data_dir_0,  data_dir_label_1=data_dir_1,  neg_pos_ratio=1)
len(dataset)

572

In [5]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs pairs: {len(dataset)}')

data = dataset[0]  # Get the first graph object.
print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes in each: {data.num_nodes}')
print(f'Number of node features: {data.num_node_features}')


Dataset: PatchDataset(572):
Number of graphs pairs: 572

PairData(adj1=[100, 100], x1=[100, 3], adj2=[100, 100], x2=[100, 3], y=0)
Number of nodes in each: None
Number of node features: 0


c:\Users\david\pyproj\pyg\pyg_env\lib\site-packages\torch_geometric\data\storage.py:271: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'x1', 'y', 'adj1', 'x2', 'adj2'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [6]:
data.x2

tensor([[ 0.2600,  0.0000, -0.9656],
        [ 0.9159,  0.0000, -0.4015],
        [ 0.3376,  0.0000, -0.9413],
        [ 0.1168,  0.0000,  0.9932],
        [-0.3257, -0.0268,  0.9451],
        [ 0.1749,  0.0000, -0.9846],
        [ 0.2071,  0.0000, -0.9783],
        [ 0.4701,  0.3119,  0.8257],
        [-0.6824, -0.1672, -0.7116],
        [-0.2098, -0.0515,  0.9764],
        [ 0.4307,  0.0000,  0.9025],
        [-0.5590, -0.0224, -0.8289],
        [ 0.1548,  0.0000,  0.9879],
        [ 0.0597,  0.0000, -0.9982],
        [-0.2739,  0.0000, -0.9618],
        [ 0.0963,  0.0000, -0.9954],
        [ 0.2348,  0.0000, -0.9720],
        [-0.0618,  0.0000, -0.9981],
        [ 0.5539,  0.0000, -0.8326],
        [ 0.1218,  0.0000, -0.9925],
        [-0.3823, -0.0201, -0.9238],
        [ 0.1407,  0.0000, -0.9901],
        [ 0.1254,  0.0000, -0.9921],
        [ 0.1351,  0.0000, -0.9908],
        [ 0.0394,  0.0000, -0.9992],
        [ 0.3158,  0.0041, -0.9488],
        [ 0.1991,  0.0000, -0.9800],
 

In [7]:
# Does not work we do not have pos
#visualize_points(data.pos, data.edge_index)

In [8]:
from torch_geometric.loader import DataLoader 

batch_size = 1

n_train = math.ceil((4/6) * len(dataset))
n_val = math.ceil((len(dataset) - n_train)/2)
n_test = len(dataset) - n_train - n_val

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])
print(f'Number of training graphs pairs: {len(train_dataset)}')
print(f'Number of validation graphs: {len(val_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

train_loader = DataLoader(dataset = train_dataset, batch_size= batch_size, shuffle=True)
val_loader = DataLoader(dataset = val_dataset, batch_size= batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size= batch_size, shuffle=True)

Number of training graphs pairs: 382
Number of validation graphs: 95
Number of test graphs: 95


In [9]:
databatch = next(iter(train_loader))
databatch

PairDataBatch(adj1=[100, 100], x1=[100, 3], adj2=[100, 100], x2=[100, 3], y=[1])

## Define Network

In [10]:
class GNN(torch.nn.Module):
    def __init__(self, in_nodes, in_channels, hidden_channels, out_channels,
                 normalize=False, lin=True):
        super(GNN, self).__init__()

        # Each instance of this GNN will have 3 convolutional layers and three batch norm layers        
        self.conv1 = DenseGCNConv(in_channels, hidden_channels, normalize)
        self.bns1 = torch.nn.BatchNorm1d(in_nodes)
        
        self.conv2 = DenseGCNConv(hidden_channels, hidden_channels, normalize)
        self.bns2 = torch.nn.BatchNorm1d(in_nodes)
        
        self.conv3 = DenseGCNConv(hidden_channels, out_channels, normalize)
        self.bns3 = torch.nn.BatchNorm1d(in_nodes)


    def forward(self, x, adj, mask=None):
        
        #Step 1
        x = self.conv1(x, adj, mask)
        x = self.bns1(x)
        
        #Step 2
        x = self.conv2(x, adj, mask)
        x = self.bns2(x)

        #Step 3
        x = self.conv3(x, adj, mask)
        if x.shape[2] != 1: 
            x = self.bns3(x)

        return x


class DiffPool(torch.nn.Module):
    def __init__(self, num_nodes):
        super(DiffPool, self).__init__()

        #Hierarchical Step #1
        in_nodes = num_nodes
        out_nodes = 25 # Number of clusters / nodes in the next layer
        self.gnn1_pool = GNN(in_nodes, dataset.num_features, 16, out_nodes) # PoolGNN --> Cluster Assignment Matrix to reduce to num_nodes
        self.gnn1_embed = GNN(in_nodes, dataset.num_features, 8, 8) # EmbGNN --> Convolutions to create new node embedding

        # Hierarchical Step #2
        in_nodes = out_nodes
        out_nodes = 10
        self.gnn2_pool = GNN(in_nodes, 8, 8, out_nodes)
        self.gnn2_embed = GNN(in_nodes, 8, 12, 16, lin=False)

        # Hierarchical Step #3
        in_nodes = out_nodes
        out_nodes = 1
        self.gnn3_pool = GNN(in_nodes, 16, 16, out_nodes)
        self.gnn3_embed = GNN(in_nodes, 16, 16, 32, lin=False)

        # Final Classifier
        self.lin1 = torch.nn.Linear(32, 64) 



    def forward(self, x, adj, mask = None, batch = None):
        
        #Hierarchical Step #1
        x1 = self.gnn1_embed(x, adj, mask) # node feature embedding
        s = self.gnn1_pool(x, adj, mask) # cluster assignment matrix

        x, adj, l1, e1 = dense_diff_pool(x1, adj, s, mask) # does the necessary matrix multiplications
        adj = torch.softmax(adj, dim=-1)


        # Hierarchical Step #2
        x2 = self.gnn2_embed(x, adj)
        s = self.gnn2_pool(x, adj)

        x, adj, l2, e2 = dense_diff_pool(x2, adj, s)
        adj = torch.softmax(adj, dim=-1)

      
        # Hierarchical Step #3
        x3 = self.gnn3_embed(x, adj)
        s = self.gnn3_pool(x, adj)
        
        x, adj, l3, e3 = dense_diff_pool(x3, adj, s)
        adj = torch.softmax(adj, dim=-1)     
        

        x = x.mean(dim=1) 

        x = F.relu(self.lin1(x))

        
        return x

### Contrastive Loss Function

In [11]:
# An attempt of a contrastive loss function
#   pairs with label 1 --> should get small euclid dist = small loss
#   pairs with label 0 --> should get large euclid dist = small loss

class ContrastiveLoss(torch.nn.Module):

    def __init__(self, margin=1):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, x0, x1, y):

        diff = x0 - x1
        pow = torch.pow(diff, 2)
        dist_sq = torch.sum(pow, 1)
        dist = torch.sqrt(dist_sq)

        mdist = self.margin- dist

        dist_marg = torch.clamp(mdist, min=0.0) # only distances <margin will be still positive here
        loss =  y * torch.pow(dist, 2) + (1-y) * torch.pow(dist_marg,2)

        return loss

### Training

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = DiffPool(num_nodes = 100).to(device)
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output1 = model(data.x1, data.adj1,None)
        output2 = model(data.x2, data.adj2, batch=None)
        
        #Contrastive Loss
        loss_contrastive = criterion(output1, output2, data.y)
        loss_contrastive.backward()
        loss_all += data.y.size(0) * loss_contrastive.item()
        optimizer.step()

    return loss_all / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    distances_lab1 = []
    distances_lab0 = []
    loss_all = 0
    
    for data in loader:
        data = data.to(device)
        output1 = model(data.x1, data.adj1, batch=None)
        output2 = model(data.x2, data.adj2, batch=None)

        loss = criterion(output1, output2, data.y)
        euclidean_distance = F.pairwise_distance(output1, output2)
        label = data.y

        if int(label) == 1: 
            distances_lab1.append(float(euclidean_distance))
            loss_all += loss.item()
        else:
            distances_lab0.append(float(euclidean_distance))
            loss_all += loss.item()

    return  distances_lab0, distances_lab1, loss_all/(len(loader))



# Initialize data collections
#==============================================================================================================================

train_distances_lab0 = []
train_distances_lab1 = []
train_losses = []

validation_distances_lab0 = []
validation_distances_lab1 = []
validation_losses = []

test_distances_lab0 = []
test_distances_lab1 = []
test_losses = []

val_dists_0 = []
val_dists_1 = []
train_dists_0 = []
train_dists_1 = []

with open(f'{model_name}_printout.txt', 'w') as f:
    f.write(f'Training Data ({model_name}):\n')
    f.close()
#==============================================================================================================================



# Test and Validation Data before Training
#==============================================================================================================================
train_results = test(train_loader)
train_distances_lab0.append(train_results[0])
train_distances_lab1.append(train_results[1])
train_losses.append(train_results[2])

validation_results = test(val_loader)
validation_distances_lab0.append(validation_results[0])
validation_distances_lab1.append(validation_results[1])
validation_losses.append(validation_results[2])

test_results = test(test_loader)
test_distances_lab0.append(test_results[0])
test_distances_lab1.append(test_results[1])
test_losses.append(test_results[2])

min_val_loss = (validation_results[2]+test_results[2])/2
#==============================================================================================================================


# Compute Euclidean Distances for printout
#==============================================================================================================================
val_dist_0 = test_results[0] + validation_results[0] # List of all euclidean distances of label 0 members of val and test dataset
val_dist_1 = test_results[1] + validation_results[1] # List of all euclidean distances of label 1 members of val and test dataset
# Ratio (mean label 0 euclid dist in validation set / mean label 1 euclid dist in validation set)
val_dist_ratio = (sum(val_dist_0) / len(val_dist_0)) / (sum(val_dist_1) / len(val_dist_1))
best_dist_ratio = val_dist_ratio
val_dists_0.append(sum(val_dist_0) / len(val_dist_0))
val_dists_1.append(sum(val_dist_1) / len(val_dist_1))

train_dist_0 = train_results[0] # List of all euclidean distances of label 0 members of train dataset
train_dist_1 = train_results[1] # List of all euclidean distances of label 0 members of train dataset
# Ratio of (mean label 0 euclid dist in training set / mean label 1 euclid dist in training set)
train_dist_ratio = (sum(train_dist_0) / len(train_dist_0)) / (sum(train_dist_1) / len(train_dist_1))
train_dists_0.append(sum(train_dist_0) / len(train_dist_0))
train_dists_1.append(sum(train_dist_1) / len(train_dist_1))
#===============================================================================================================================

printout = (f'Before Tr:  Train Loss: {train_results[2]:.3f}, Val Loss: {validation_results[2]:.3f}, Test Loss: {test_results[2]:.3f}, '
            f'ValDistanceRatio (L0/L1): {(val_dist_ratio):.3f}, TrainDistanceRatio (L0/L1): {(train_dist_ratio):.3f}')

print(printout)

# append to outputs txt
with open(f'{model_name}_printout.txt', 'a') as f:
        f.write(printout + '\n')
        f.close()



#===============================================================================================================================================
# Training
#===============================================================================================================================================

for epoch in range(1, num_epochs+1):
    
    train_loss = train(epoch)


    # Test and Validation Data During Training
    #===============================================================================================================================================
    train_results = test(train_loader)
    train_distances_lab0.append(train_results[0])
    train_distances_lab1.append(train_results[1])
    train_losses.append(train_results[2])

    validation_results = test(val_loader)
    validation_distances_lab0.append(validation_results[0])
    validation_distances_lab1.append(validation_results[1])
    validation_losses.append(validation_results[2])

    test_results = test(test_loader)
    test_distances_lab0.append(test_results[0])
    test_distances_lab1.append(test_results[1])
    test_losses.append(test_results[2])

    #===============================================================================================================================================


    # Compute Euclidean Distances for printout
    #==============================================================================================================================
    val_dist_0 = test_results[0]+validation_results[0] # List of all euclidean distances of label 0 members of val and test dataset
    val_dist_1 = test_results[1]+validation_results[1] # List of all euclidean distances of label 1 members of val and test dataset
    # Ratio (mean label 0 euclid dist in validation set / mean label 1 euclid dist in validation set)
    val_dist_ratio = (sum(val_dist_0) / len(val_dist_0)) / (sum(val_dist_1) / len(val_dist_1))
    val_dists_0.append(sum(val_dist_0) / len(val_dist_0))
    val_dists_1.append(sum(val_dist_1) / len(val_dist_1))

    train_dist_0 = train_results[0] # List of all euclidean distances of label 0 members of train dataset
    train_dist_1 = train_results[1] # List of all euclidean distances of label 0 members of train dataset
    # Ratio of (mean label 0 euclid dist in training set / mean label 1 euclid dist in training set)
    train_dist_ratio = (sum(train_dist_0) / len(train_dist_0)) / (sum(train_dist_1) / len(train_dist_1))
    train_dists_0.append(sum(train_dist_0) / len(train_dist_0))
    train_dists_1.append(sum(train_dist_1) / len(train_dist_1))
    #===============================================================================================================================
    


    # Create Printout
    #===============================================================================================================================================
    #printout = (f'Epoch: {epoch:03d}, Train Loss: {train_results[2]:.3f}, Val Loss: {validation_results[2]:.3f}, Test Loss: {test_results[2]:.3f}, '
    #        f'ValDist 0: {val_dist_0:.3f}, ValDist 1: {val_dist_1:.3f}, TrainDist 0: {train_dist_0:.3f}, TrainDist 1: {train_dist_1:.3f}')
    printout = (f'Epoch: {epoch:03d}, Train Loss: {train_results[2]:.3f}, Val Loss: {validation_results[2]:.3f}, Test Loss: {test_results[2]:.3f}, '
            f'ValDistanceRatio (L0/L1): {(val_dist_ratio):.3f}, TrainDistanceRatio (L0/L1): {(train_dist_ratio):.3f}')
    print(printout)

    # append to outputs txt
    with open(f'{model_name}_printout.txt', 'a') as f:
        f.write(printout + '\n')
        f.close()
    #===============================================================================================================================================



    # Save the model outputs
    #===============================================================================================================================================
    np.save(f'{model_name}_training_loss.npy', train_losses, allow_pickle=True)
    np.save(f'{model_name}_training_distances_lab0.npy', train_distances_lab0, allow_pickle=True)
    np.save(f'{model_name}_training_distances_lab1.npy', train_distances_lab1, allow_pickle=True)

    np.save(f'{model_name}_validation_loss.npy', validation_losses, allow_pickle=True)
    np.save(f'{model_name}_validation_distances_lab0.npy', validation_distances_lab0, allow_pickle=True)
    np.save(f'{model_name}_validation_distances_lab1.npy', validation_distances_lab1, allow_pickle=True)

    np.save(f'{model_name}_test_loss.npy', test_losses, allow_pickle=True)
    np.save(f'{model_name}_test_distances_lab0.npy', test_distances_lab0, allow_pickle=True)
    np.save(f'{model_name}_test_distances_lab1.npy', test_distances_lab1, allow_pickle=True)
    
    last_saved_epoch = epoch
    #===============================================================================================================================================


    # If the epoch has improved the model, save it
    if val_dist_ratio > best_dist_ratio:
        new_best = True
        best_dist_ratio = val_dist_ratio
        best_epoch = epoch
    else:
        new_best = False
        
    if new_best:
        torch.save(model.state_dict(), f'{model_name}_state_dict_best.pt')
        torch.save(model, f'{model_name}_save.pt')

cpu
Before Tr:  Train Loss: 233.268, Val Loss: 170.849, Test Loss: 223.307, ValDistanceRatio (L0/L1): 1.375, TrainDistanceRatio (L0/L1): 1.395
Epoch: 001, Train Loss: 409.001, Val Loss: 328.655, Test Loss: 396.247, ValDistanceRatio (L0/L1): 1.182, TrainDistanceRatio (L0/L1): 1.210
Epoch: 002, Train Loss: 268.490, Val Loss: 72.423, Test Loss: 150.844, ValDistanceRatio (L0/L1): 1.884, TrainDistanceRatio (L0/L1): 1.653
Epoch: 003, Train Loss: 359.715, Val Loss: 265.609, Test Loss: 312.821, ValDistanceRatio (L0/L1): 1.017, TrainDistanceRatio (L0/L1): 0.981
Epoch: 004, Train Loss: 143.339, Val Loss: 76.576, Test Loss: 111.503, ValDistanceRatio (L0/L1): 1.332, TrainDistanceRatio (L0/L1): 1.313
Epoch: 005, Train Loss: 170.992, Val Loss: 155.723, Test Loss: 177.429, ValDistanceRatio (L0/L1): 1.035, TrainDistanceRatio (L0/L1): 1.037
Epoch: 006, Train Loss: 143.491, Val Loss: 41.588, Test Loss: 81.323, ValDistanceRatio (L0/L1): 1.741, TrainDistanceRatio (L0/L1): 1.569
Epoch: 007, Train Loss: 286

KeyboardInterrupt: 

### Evaluation

In [ ]:
best_epoch, last_saved_epoch

In [ ]:
import matplotlib.pyplot as plt

def compare_euclid_distances(distances_lab0, distances_lab1, save=None, title=None):

    '''Function for plotting the euclidean distances of two datasets generated during a model epoch'''

    w = 0.8    # bar width
    x = [1, 2] # x-coordinates of your bars
    colors = [(0, 0, 1, 1), (1, 0, 0, 1)]    # corresponding colors

    # Epoch 0
    y = [distances_lab0, distances_lab1]


    fig, ax = plt.subplots()
    ax.bar(x,
        height=[np.mean(yi) for yi in y],
        yerr=[np.std(yi) for yi in y],    # error bars
        capsize=12, # error bar cap width in points
        width=w,    # bar width
        tick_label=["Label 0", "Label 1"],
        color=(0,0,0,0),  # face color transparent
        edgecolor=colors,
        )

    for i in range(len(x)):
        # distribute scatter randomly across whole width of bar
        ax.scatter(x[i] + np.random.random(len(y[i])) * w - w / 2, y[i], color=colors[i])

    plt.title(title)
    plt.ylabel('Euclidean Distance')
    #plt.ylim(top=1.5, bottom=0)
    
    if save != None:
        plt.savefig(save)
    plt.show()


### Compare Euclidean Distances in the Training Set (Before and after Training)

In [ ]:
compare_euclid_distances(train_distances_lab0[0], train_distances_lab1[0], save = model_name + '_plot_train_dist0.png', title = 'Training Distances Epoch 0')
compare_euclid_distances(train_distances_lab0[best_epoch], train_distances_lab1[best_epoch], save = model_name + f'_plot_train_dist_{best_epoch}', title = f'Training Distances Epoch {best_epoch}')

### Compare Euclidean Distances in the Validation Set (Before and after Training)

In [ ]:
compare_euclid_distances(validation_distances_lab0[0], validation_distances_lab1[0], save = model_name + '_plot_val_dist0.png', title = 'Validation Distances Epoch 0')
compare_euclid_distances(validation_distances_lab0[best_epoch], validation_distances_lab1[best_epoch], save = model_name + f'_plot_val_dist_{best_epoch}', title = f'Validation Distances Epoch {best_epoch}')

### Plot Training and Validation Losses

In [ ]:
import matplotlib.pyplot as plt

# Generate a sequence of integers to represent the epoch numbers
epochs = range(1, last_saved_epoch+2)
 
# Plot and label the training and validation loss values
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, validation_losses, label='Validation Loss')
plt.plot(epochs, test_losses, label='Test Loss')
 
# Add in a title and axes labels
plt.title('Training and Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
 
# Set the tick locations
plt.xticks(np.arange(0, last_saved_epoch+1, last_saved_epoch/10))
 
# Display the plot
plt.legend(loc='best')
plt.savefig(f'{model_name}_plot_loss.png')
#plt.ylim(top=60, bottom = 0)
plt.show()


### Plot Euclidean Distances (Label0 vs. Label1)

In [ ]:
import matplotlib.pyplot as plt

# Generate a sequence of integers to represent the epoch numbers
epochs = range(1, last_saved_epoch+2)
 
# Plot and label the training and validation loss values
plt.plot(epochs, train_dists_0, label='Training Distances Label 0')
plt.plot(epochs, train_dists_1, label='Training Distances Label 1')
 
# Add in a title and axes labels
plt.title('Training Euclidean Distances')
plt.xlabel('Epochs')
plt.ylabel('Euclidean Distance')

 
# Set the tick locations
plt.xticks(np.arange(0, last_saved_epoch+1, last_saved_epoch/10))
 
# Display the plot
plt.legend(loc='best')
plt.savefig(f'{model_name}_plot_train_distances.png')
#plt.ylim(top=60, bottom = 0)
plt.show()